Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [142]:
import time
from collections import deque
from random import choice
from tqdm.asyncio import tqdm_asyncio
from tqdm.auto import tqdm
from enum import Enum
import heapq as hq
import numpy as np
import numpy.typing as npt

In [143]:
PUZZLE_DIM = 5
VALID_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

In [144]:
class Action(Enum):
    LEFT = [[0], [1]]
    RIGHT = [[0], [-1]]
    UP = [[1], [0]]
    DOWN = [[-1], [0]]

# I took inspiration for these classes from Leone Fabio's implementation.
class PuzzleState:
    def __init__(self, board: npt.NDArray[np.int_], parent_state = None, action: Action | None = None, depth: int = 0, cost: int = 0, key: int = 0) -> None:
        self.board: npt.NDArray[np.int_] = board
        self.parent_state = parent_state # type: PuzzleState | None
        self.action: Action | None = action
        self.depth: int = depth
        self.cost: int = cost
        self.key: int = key
        self.map: bytes = self.board.tobytes()

    def __lt__(self, other) -> bool:
        # Used for heapq
        return self.key < other.key

class SearchAlgorithm:
    def __init__(self, start_state: PuzzleState) -> None:
        self.start_state: PuzzleState = start_state
        self.goal_state: PuzzleState | None = None
        self.max_frontier_size: int = 0
        self.max_search_depth: int = 0
        self.expanded_nodes: int = 0
        self.path: list[Action] = []

    def reconstruct_path(self) -> None:
        state: PuzzleState = self.goal_state
        while state and state.parent_state:
            self.path.insert(0, state.action)
            state = state.parent_state

In [145]:
def is_action_valid(board: npt.NDArray[np.int_], action: Action, action_bounds: tuple[tuple[int, int], tuple[int, int]] = ((-1, -1), (PUZZLE_DIM, PUZZLE_DIM))) -> bool:
    hole_pos = np.nonzero(board == 0)
    target_pos = tuple(np.add(hole_pos, action.value))

    return not (target_pos[0] == action_bounds[0][1] or target_pos[1] == action_bounds[0][0] or target_pos[0] == action_bounds[1][1] or target_pos[1] == action_bounds[1][0])

def available_actions(board: npt.NDArray[np.int_]) -> list[Action]:
    actions: list[Action] = []

    if is_action_valid(board, Action.LEFT):
        actions.append(Action.LEFT)
    if is_action_valid(board, Action.RIGHT):
        actions.append(Action.RIGHT)
    if is_action_valid(board, Action.UP):
        actions.append(Action.UP)
    if is_action_valid(board, Action.DOWN):
        actions.append(Action.DOWN)

    return actions

In [146]:
def get_valid_positions(board: npt.NDArray[np.int_]) -> npt.NDArray[np.bool_]:
    return np.array(board == VALID_STATE)

def get_valid_rows(board: npt.NDArray[np.int_]) -> int:
    if np.argwhere(np.all(get_valid_positions(board), axis=1) == 0).shape == (0, 1):
        return PUZZLE_DIM

    return int(np.cumsum(np.argwhere(np.all(get_valid_positions(board), axis=1) == 0))[0])

def get_valid_columns(board: npt.NDArray[np.int_]) -> int:
    if np.argwhere(np.all(get_valid_positions(board), axis=0) == 0).shape == (0, 1):
        return PUZZLE_DIM

    return int(np.cumsum(np.argwhere(np.all(get_valid_positions(board), axis=0) == 0))[0])

def is_valid(board: npt.NDArray[np.int_]) -> bool:
    return get_valid_rows(board) == PUZZLE_DIM and get_valid_columns(board) == PUZZLE_DIM

In [147]:
def do_action(board: npt.NDArray[np.int_], action: Action, action_bounds: tuple[tuple[int, int], tuple[int, int]] = ((-1, -1), (PUZZLE_DIM, PUZZLE_DIM))) -> np.ndarray:
    if not is_action_valid(board, action, action_bounds):
        return board

    new_state = board.copy()

    hole_pos = np.nonzero(new_state == 0)
    target_pos = tuple(np.add(hole_pos, action.value))

    new_state[hole_pos], new_state[target_pos] = new_state[target_pos], new_state[hole_pos]
    return new_state

## Search algorithms

These are the 3 main algorithms that are generally used to solve path-searching problems:

- Breadth-first search (BFS): it starts at the initial state and explores all the states at the same depth level, before moving to the next depth level, and so on until it finds the correct solution.
- Depth-first search (DFS): its starts at the initial state and explores as deep as possible before backtracking, continuing until it finds the correct solution.
- A*: uses a defined heuristic (i.e. a function that returns a "distance" metric between two states) to search for the shortest path between the initial state and a defined goal state.

Generally speaking, BFS and DFS have often similar costs (i.e. number of states explored before finding a solution) and  relatively high run times (with respect to the complexity of the puzzle), with BFS having slightly better performances.
A*, on the other hand, has much better performances, with respectable run times and low costs.

In [148]:
# Breadth first search
class BFS(SearchAlgorithm):
    def run(self) -> None:
        visited_boards: set[bytes] = set()
        queue: deque[PuzzleState] = deque([PuzzleState(self.start_state.board)])
        progress: tqdm_asyncio = tqdm(total=1, desc="BFS progress", unit="node", dynamic_ncols=True)

        while queue:
            state: PuzzleState = queue.popleft()
            visited_boards.add(state.map)
            self.expanded_nodes += 1
            state.cost = self.expanded_nodes
            progress.update(1)

            if np.array_equal(state.board, VALID_STATE):
                progress.close()
                self.goal_state = state
                self.reconstruct_path()
                return

            for action in available_actions(state.board):
                new_board: npt.NDArray[np.int_] = do_action(state.board, action)
                new_state: PuzzleState = PuzzleState(new_board, parent_state=state, action=action, depth=state.depth + 1)

                if new_state.map not in visited_boards:
                    queue.append(new_state)
                    visited_boards.add(new_state.map)
                    self.max_frontier_size = max(self.max_frontier_size, len(queue))
                    self.max_search_depth = max(self.max_search_depth, new_state.depth)

        progress.close()

# Depth first search
class DFS(SearchAlgorithm):
    def run(self) -> None:
        visited_boards: set[bytes] = set()
        stack: list[PuzzleState] = [PuzzleState(self.start_state.board)]
        progress: tqdm_asyncio = tqdm(total=1, desc="DFS progress", dynamic_ncols=True)

        while stack:
            state: PuzzleState = stack.pop()
            visited_boards.add(state.map)
            self.expanded_nodes += 1
            state.cost = self.expanded_nodes
            progress.update(1)

            if np.array_equal(state.board, VALID_STATE):
                progress.close()
                self.goal_state = state
                self.reconstruct_path()
                return

            for action in available_actions(state.board):
                new_board: npt.NDArray[np.int_] = do_action(state.board, action)
                new_state: PuzzleState = PuzzleState(new_board, parent_state=state, action=action, depth=state.depth + 1)

                if new_state.map not in visited_boards:
                    stack.append(new_state)
                    visited_boards.add(new_state.map)
                    self.max_frontier_size = max(self.max_frontier_size, len(stack))
                    self.max_search_depth = max(self.max_search_depth, new_state.depth)

        progress.close()

# A*
def manhattan_distance(board: npt.NDArray[np.int_]) -> int:
    distance: int = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            value = board[i, j]

            if value != 0:
                target_x, target_y = divmod(value - 1, PUZZLE_DIM)
                distance += abs(target_x - i) + abs(target_y - j)

    return distance

class AStar(SearchAlgorithm):
    def run(self) -> None:
        visited_boards: set[bytes] = set()
        initial_key: int = manhattan_distance(self.start_state.board)
        queue: list[PuzzleState] = [PuzzleState(self.start_state.board, key=initial_key)]
        hq.heapify(queue)
        progress: tqdm_asyncio = tqdm(total=1, desc="AStar progress", dynamic_ncols=True)

        while queue:
            state: PuzzleState = hq.heappop(queue)
            self.expanded_nodes += 1
            state.cost = self.expanded_nodes
            progress.update(1)

            if np.array_equal(state.board, VALID_STATE):
                progress.close()
                self.goal_state = state
                self.reconstruct_path()
                return

            for action in available_actions(state.board):
                new_board: npt.NDArray[np.int_] = do_action(state.board, action)
                new_state: PuzzleState = PuzzleState(new_board, parent_state=state, action=action, depth=state.depth + 1)

                if new_state.map not in visited_boards:
                    new_state.key = manhattan_distance(new_state.board)
                    hq.heappush(queue, new_state)
                    visited_boards.add(new_state.map)
                    self.max_frontier_size = max(self.max_frontier_size, len(queue))
                    self.max_search_depth = max(self.max_search_depth, new_state.depth)

        progress.close()

## Algorithmic solver.

Instead of searching for a path, this solver uses specific actions sequences that guarantee to solve the puzzle (if it's solvable).

This algorithm does not need to "explore" any board configuration to find the solution, thus its maximum depth will always be 1 and the cost of its solution will always be equal to the length of the final path.

Due to its deterministic nature, this solver vastly outclasses the 3 main search algorithms for this specific problem (NxN tiling puzzle), but naturally cannot be used as-is for any other path-search problem.

In [149]:
def is_target_at_correct_position(board: npt.NDArray[np.int_], target: int) -> bool:
    return np.all(np.argwhere(board == target) == np.argwhere(VALID_STATE == target))

def place_target_in_position(state: PuzzleState, target: int, position: tuple[int, int], movement_bounds: tuple[tuple[int, int], tuple[int, int]] = ((-1, -1), (PUZZLE_DIM, PUZZLE_DIM)), verbose: bool = False) -> PuzzleState:
    if is_target_at_correct_position(state.board, target):
        return state

    current_state: PuzzleState = state
    new_board: npt.NDArray[np.int_]

    target_pos: npt.NDArray[np.int_] = np.array(position[::-1])
    if verbose:
        print('target_pos', target_pos)
        print('aligning gap')

    # Place gap at the edge of the region defined by the target current position and its correct position.
    while True:
        gap_position: tuple = tuple(np.argwhere(current_state.board == 0).flatten())[::-1]
        current_position: tuple = tuple(np.argwhere(current_state.board == target).flatten())[::-1]
        if (
            gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
            gap_position[0] == position[0] or gap_position[1] == position[1] or
            is_target_at_correct_position(current_state.board, target)
        ):
            break

        hole_pos: npt.NDArray[np.int_] = np.argwhere(current_state.board == 0).flatten()
        starting_pos: npt.NDArray[np.int_] = np.argwhere(current_state.board == target).flatten()

        if verbose:
            print(current_state.board)
            print('hole_pos', hole_pos)
            print('starting_pos', starting_pos)

        positions: npt.NDArray[np.int_] = np.array([hole_pos, starting_pos]).reshape(2, 2)

        min_values: npt.NDArray[np.int_] = np.min(positions, axis=0)
        max_values: npt.NDArray[np.int_] = np.max(positions, axis=0)

        min_y, min_x = tuple(min_values)
        max_y, max_x = tuple(max_values)

        min_x -= 1
        min_y -= 1
        max_x += 1
        max_y += 1

        if verbose:
            print('top_left', (min_x, min_y))
            print('bottom_right', (max_x, max_y))

        if min_x < movement_bounds[0][0]:
            min_x = movement_bounds[0][0]
        if min_y < movement_bounds[0][1]:
            min_y = movement_bounds[0][1]
        if max_x > movement_bounds[1][0]:
            max_x = movement_bounds[1][0]
        if max_y > movement_bounds[1][1]:
            max_y = movement_bounds[1][1]

        if max_x - min_x == 2:
            if max_x == PUZZLE_DIM:
                min_x -= 1
            else:
                max_x += 1
        if max_y - min_y == 2:
            if max_y == PUZZLE_DIM:
                min_y -= 1
            else:
                max_y += 1

        bounds: tuple[tuple[int, int], tuple[int, int]] = ((min_x, min_y), (max_x, max_y))
        if verbose:
            print('movement_bounds', movement_bounds)
            print('bounds', bounds)

        if not is_action_valid(current_state.board, Action.DOWN, bounds):
            new_board = do_action(current_state.board, Action.RIGHT, bounds)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.RIGHT, cost=current_state.cost + 1)
            if verbose:
                print(current_state.board)

            gap_position = tuple(np.argwhere(current_state.board == 0).flatten())[::-1]
            current_position = tuple(np.argwhere(current_state.board == target).flatten())[::-1]
            if (
                gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
                gap_position[0] == position[0] or gap_position[1] == position[1] or
                is_target_at_correct_position(new_board, target)
            ):
                break

        if not is_action_valid(current_state.board, Action.LEFT, bounds):
            new_board = do_action(current_state.board, Action.DOWN, bounds)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.DOWN, cost=current_state.cost + 1)
            if verbose:
                print(current_state.board)

            gap_position = tuple(np.argwhere(current_state.board == 0).flatten())[::-1]
            current_position = tuple(np.argwhere(current_state.board == target).flatten())[::-1]
            if (
                gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
                gap_position[0] == position[0] or gap_position[1] == position[1] or
                is_target_at_correct_position(new_board, target)
            ):
                break

        if not is_action_valid(current_state.board, Action.UP, bounds):
            new_board = do_action(current_state.board, Action.LEFT, bounds)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)
            if verbose:
                print(current_state.board)

            gap_position = tuple(np.argwhere(current_state.board == 0).flatten())[::-1]
            current_position = tuple(np.argwhere(current_state.board == target).flatten())[::-1]
            if (
                gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
                gap_position[0] == position[0] or gap_position[1] == position[1] or
                is_target_at_correct_position(new_board, target)
            ):
                break

        if not is_action_valid(current_state.board, Action.RIGHT, bounds):
            new_board = do_action(current_state.board, Action.UP, bounds)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)
            if verbose:
                print(current_state.board)

            gap_position = tuple(np.argwhere(current_state.board == 0).flatten())[::-1]
            current_position = tuple(np.argwhere(current_state.board == target).flatten())[::-1]
            if (
                gap_position[0] == current_position[0] or gap_position[1] == current_position[1] or
                gap_position[0] == position[0] or gap_position[1] == position[1] or
                is_target_at_correct_position(new_board, target)
            ):
                break
    if verbose:
        print('gap aligned')
        print('aligning_target')

    # Place target in position
    while True:
        hole_pos: npt.NDArray[np.int_] = np.argwhere(current_state.board == 0).flatten()
        starting_pos: npt.NDArray[np.int_] = np.argwhere(current_state.board == target).flatten()

        if verbose:
            print(current_state.board)
            print('hole_pos', hole_pos)
            print('starting_pos', starting_pos)
            print('target_pos', target_pos)

        positions: npt.NDArray[np.int_] = np.array([hole_pos, starting_pos, target_pos]).reshape(3, 2)

        min_values: npt.NDArray[np.int_] = np.min(positions, axis=0)
        max_values: npt.NDArray[np.int_] = np.max(positions, axis=0)

        min_y, min_x = tuple(min_values)
        max_y, max_x = tuple(max_values)

        min_x -= 1
        min_y -= 1
        max_x += 1
        max_y += 1

        if verbose:
            print('top_left', (min_x, min_y))
            print('bottom_right', (max_x, max_y))

        if min_x < movement_bounds[0][0]:
            min_x = movement_bounds[0][0]
        if min_y < movement_bounds[0][1]:
            min_y = movement_bounds[0][1]
        if max_x > movement_bounds[1][0]:
            max_x = movement_bounds[1][0]
        if max_y > movement_bounds[1][1]:
            max_y = movement_bounds[1][1]

        if max_x - min_x == 2:
            if max_x == PUZZLE_DIM:
                min_x -= 1
            else:
                max_x += 1
        if max_y - min_y == 2:
            if max_y == PUZZLE_DIM:
                min_y -= 1
            else:
                max_y += 1

        bounds: tuple[tuple[int, int], tuple[int, int]] = ((min_x, min_y), (max_x, max_y))
        if verbose:
            print('movement_bounds', movement_bounds)
            print('bounds', bounds)

        if not is_action_valid(current_state.board, Action.DOWN, bounds):
            new_board = do_action(current_state.board, Action.RIGHT, bounds)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.RIGHT, cost=current_state.cost + 1)
            if verbose:
                print(current_state.board)

            current_position = tuple(np.argwhere(current_state.board == target).flatten())[::-1]
            if current_position == position or is_target_at_correct_position(new_board, target):
                break

        if not is_action_valid(current_state.board, Action.LEFT, bounds):
            new_board = do_action(current_state.board, Action.DOWN, bounds)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.DOWN, cost=current_state.cost + 1)
            if verbose:
                print(current_state.board)

            current_position = tuple(np.argwhere(current_state.board == target).flatten())[::-1]
            if current_position == position or is_target_at_correct_position(new_board, target):
                break

        if not is_action_valid(current_state.board, Action.UP, bounds):
            new_board = do_action(current_state.board, Action.LEFT, bounds)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)
            if verbose:
                print(current_state.board)

            current_position = tuple(np.argwhere(current_state.board == target).flatten())[::-1]
            if current_position == position or is_target_at_correct_position(new_board, target):
                break

        if not is_action_valid(current_state.board, Action.RIGHT, bounds):
            new_board = do_action(current_state.board, Action.UP, bounds)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)
            if verbose:
                print(current_state.board)

            current_position = tuple(np.argwhere(current_state.board == target).flatten())[::-1]
            if current_position == position or is_target_at_correct_position(new_board, target):
                break

    if verbose:
        print('target_aligned')

    return current_state

def place_target_in_correct_position(state: PuzzleState, target: int, verbose=False) -> PuzzleState:
    target_y, target_x = tuple(np.argwhere(VALID_STATE == target).flatten())
    if is_target_at_correct_position(state.board, target):
        return state

    building_row: bool = (get_valid_rows(state.board) == target_y and target_x != 0) or (target_x, target_y) == (0, 0)

    solving_region: tuple[tuple[int, int], tuple[int, int]] = ((target_x - 1, target_y - 1), (PUZZLE_DIM, PUZZLE_DIM))
    pivoting_region: tuple[tuple[int, int], tuple[int, int]] = ((get_valid_columns(state.board) - 1 + int(not building_row), get_valid_rows(state.board) - 1 + int(building_row and target_x != 0)), (PUZZLE_DIM, PUZZLE_DIM))

    current_state: PuzzleState = state
    new_board: npt.NDArray[np.int_]

    if verbose:
        print('target', target)
        print('solving_region', solving_region)
        print('pivoting_region', pivoting_region)
        print('building_row', building_row)

    if target_x == PUZZLE_DIM - 1 or target_y == PUZZLE_DIM - 1:
        # Corner case

        # Position gap inside solving region
        new_board = do_action(current_state.board, Action.UP)
        current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)
        if is_target_at_correct_position(current_state.board, target):
            return current_state

        new_board = do_action(current_state.board, Action.LEFT)
        current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)
        if is_target_at_correct_position(current_state.board, target):
            return current_state

        offset: tuple[int, int] = (-1, 1) if target_x == PUZZLE_DIM - 1 else (1, -1)
        pivot = tuple(np.add((target_x, target_y), offset))

        if verbose:
            print('pivot', pivot)

        # Placing target at pivot
        current_state = place_target_in_position(current_state, target, pivot, pivoting_region, verbose)

        # Placing gap at row below/column to the right of the pivot (building row/column)
        while True:
            gap_y, gap_x = tuple(np.argwhere(current_state.board == 0).flatten())

            if verbose:
                print(current_state.board)

            if building_row:
                if gap_y == pivot[1] + 1:
                    break

                if verbose:
                    print('positioning gap in row below target')

                if gap_y < pivot[1] + 1:
                    new_board = do_action(current_state.board, Action.UP)
                    current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)
                else:
                    new_board = do_action(current_state.board, Action.DOWN)
                    current_state = PuzzleState(new_board, parent_state=current_state, action=Action.DOWN, cost=current_state.cost + 1)
            else:
                if gap_x == pivot[0] + 1:
                    break

                if verbose:
                    print('positioning gap in column to the right of target')

                if gap_x < pivot[0] + 1:
                    new_board = do_action(current_state.board, Action.LEFT)
                    current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)
                else:
                    new_board = do_action(current_state.board, Action.RIGHT)
                    current_state = PuzzleState(new_board, parent_state=current_state, action=Action.RIGHT, cost=current_state.cost + 1)

        # Placing gap in column to the left/row above the pivot (building row/column)
        while True:
            gap_y, gap_x = tuple(np.argwhere(current_state.board == 0).flatten())

            if verbose:
                print(current_state.board)

            if building_row and gap_x == pivot[0] - 1 or not building_row and gap_y == pivot[1] - 1:
                break

            if verbose:
                print('adjusting gap')

            if building_row:
                new_board = do_action(current_state.board, Action.RIGHT)
                current_state = PuzzleState(new_board, parent_state=current_state, action=Action.RIGHT, cost=current_state.cost + 1)
            else:
                new_board = do_action(current_state.board, Action.DOWN)
                current_state = PuzzleState(new_board, parent_state=current_state, action=Action.DOWN, cost=current_state.cost + 1)

        # Gap final placement
        if building_row:
            new_board = do_action(current_state.board, Action.DOWN)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.DOWN, cost=current_state.cost + 1)
        else:
            new_board = do_action(current_state.board, Action.RIGHT)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.RIGHT, cost=current_state.cost + 1)

        # Fixed movements
        if building_row:
            new_board = do_action(current_state.board, Action.DOWN)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.DOWN, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.LEFT)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.UP)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.LEFT)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.DOWN)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.DOWN, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.RIGHT)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.RIGHT, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.RIGHT)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.RIGHT, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.UP)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)
        else:
            new_board = do_action(current_state.board, Action.RIGHT)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.RIGHT, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.UP)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.LEFT)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.UP)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.RIGHT)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.RIGHT, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.DOWN)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.DOWN, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.DOWN)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.DOWN, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.LEFT)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)

        if pivot == (PUZZLE_DIM - 2, PUZZLE_DIM - 2) and not building_row:
            new_board = do_action(current_state.board, Action.LEFT)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)
            new_board = do_action(current_state.board, Action.UP)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)

        return current_state

    current_y, current_x = tuple(np.argwhere(current_state.board == target).flatten())
    if current_x <= solving_region[0][0] or current_y <= solving_region[0][1]:
        # Position gap inside pivoting region
        while True:
            gap_y, gap_x = tuple(np.argwhere(current_state.board == 0).flatten())

            if verbose:
                print(current_state.board)

            if gap_x > pivoting_region[0][0] and gap_y > pivoting_region[0][1]:
                if verbose:
                    print('gap inside pivoting region')

                break

            if verbose:
                print('placing gap inside pivoting region')

            new_board = do_action(current_state.board, Action.UP)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)
            if is_target_at_correct_position(current_state.board, target):
                return current_state

            new_board = do_action(current_state.board, Action.LEFT)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)
            if is_target_at_correct_position(current_state.board, target):
                return current_state

        # Position target directly below/to the right of its correct position (row/column)
        if verbose:
            print('positioning target inside solving region')

        offset = (int(not building_row), int(building_row))
        target_pos = tuple(np.add((target_x, target_y), offset))

        if verbose:
            print('target_pos', target_pos)

        current_state = place_target_in_position(current_state, target, target_pos, pivoting_region, verbose)
        if is_target_at_correct_position(current_state.board, target):
            return current_state

    if verbose:
        print('target inside solving region')

    # Position gap inside solving region
    while True:
        gap_y, gap_x = tuple(np.argwhere(current_state.board == 0).flatten())

        if verbose:
            print(current_state.board)

        if gap_x > solving_region[0][0] and gap_y > solving_region[0][1]:
            if verbose:
                print('gap inside solving region')

            break

        if verbose:
            print('placing gap inside solving region')

        new_board = do_action(current_state.board, Action.UP)
        current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)
        if is_target_at_correct_position(current_state.board, target):
            return current_state

        new_board = do_action(current_state.board, Action.LEFT)
        current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)
        if is_target_at_correct_position(current_state.board, target):
            return current_state

    current_y, current_x = tuple(np.argwhere(current_state.board == target).flatten())
    if current_x <= solving_region[0][0] or current_y <= solving_region[0][1]:
        # Position gap inside pivoting region
        while True:
            gap_y, gap_x = tuple(np.argwhere(current_state.board == 0).flatten())

            if verbose:
                print(current_state.board)

            if gap_x > pivoting_region[0][0] and gap_y > pivoting_region[0][1]:
                if verbose:
                    print('gap inside pivoting region')

                break

            if verbose:
                print('placing gap inside pivoting region')

            new_board = do_action(current_state.board, Action.UP)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)
            if is_target_at_correct_position(current_state.board, target):
                return current_state

            new_board = do_action(current_state.board, Action.LEFT)
            current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)
            if is_target_at_correct_position(current_state.board, target):
                return current_state

        # Positioning target directly below/to the right of its correct position (row/column)
        if verbose:
            print('placing target inside solving region')

        offset = (int(not building_row), int(building_row))
        target_pos = tuple(np.add((target_x, target_y), offset))

        if verbose:
            print('target_pos', target_pos)

        current_state = place_target_in_position(current_state, target, target_pos, pivoting_region, verbose)
        if is_target_at_correct_position(current_state.board, target):
            return current_state

    if verbose:
        print('target inside solving region')

    # Position gap inside solving region
    while True:
        gap_y, gap_x = tuple(np.argwhere(current_state.board == 0).flatten())

        if verbose:
            print(current_state.board)

        if gap_x > solving_region[0][0] and gap_y > solving_region[0][1]:
            if verbose:
                print('gap inside solving region')

            break

        if verbose:
            print('placing gap inside solving region')

        new_board = do_action(current_state.board, Action.UP)
        current_state = PuzzleState(new_board, parent_state=current_state, action=Action.UP, cost=current_state.cost + 1)
        if is_target_at_correct_position(current_state.board, target):
            return current_state

        new_board = do_action(current_state.board, Action.LEFT)
        current_state = PuzzleState(new_board, parent_state=current_state, action=Action.LEFT, cost=current_state.cost + 1)
        if is_target_at_correct_position(current_state.board, target):
            return current_state

    current_state = place_target_in_position(current_state, target, (target_x, target_y), solving_region, verbose)

    return current_state

class FastSolver(SearchAlgorithm):
    def run(self):
        current_state: PuzzleState = self.start_state
        progress: tqdm_asyncio = tqdm(total=PUZZLE_DIM * PUZZLE_DIM - 1, desc='FastSolver progress', unit='numbers correctly placed', dynamic_ncols=True)

        for i in range(PUZZLE_DIM):
            current_state = place_target_in_correct_position(current_state, i + 1)
            progress.update(1)

        for k in range(1, PUZZLE_DIM - 1):
            for i in range(PUZZLE_DIM - k):
                current_state = place_target_in_correct_position(current_state, k + PUZZLE_DIM * (i + k))
                progress.update(1)
            for i in range(PUZZLE_DIM - k):
                current_state = place_target_in_correct_position(current_state, k + i + 1 + PUZZLE_DIM * k)
                progress.update(1)

        current_state = place_target_in_correct_position(current_state, PUZZLE_DIM * PUZZLE_DIM - 1)
        progress.update(1)
        progress.close()

        self.goal_state = current_state

In [150]:
RANDOMIZE_STEPS = 100000

initial_board = VALID_STATE
for _ in tqdm(range(RANDOMIZE_STEPS)):
    initial_board = do_action(initial_board, choice(available_actions(initial_board)))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [151]:
initial_board

array([[20, 24, 11,  9, 16],
       [18, 23, 12, 10,  3],
       [ 7,  4,  6,  5, 13],
       [22,  0,  1, 21, 19],
       [15, 14,  8,  2, 17]])

## Results

As expected, the BFS and DFS solvers are the slowest, to the point of having decent run times only for 3x3 puzzles (they take more than 5 minutes just for 4x4 puzzles), since naturally their run time increases exponentially with higher dimension puzzles).
They also produce longer and poorly optimized paths.

The A* solver has better performances, only beginning to have noticeable increases in run time with 6x6 puzzles. The paths produced by A* are also better optimized, and sometimes (albeit very rarely) even slightly shorter than those produced by the algorithmic solver.

The algorithmic solver is by far the best one, with run times faster than 1s even for 10x10 puzzles and extremely optimized paths, slightly longer than the BFS ones (which are the theoretically shortest possible paths).

In [152]:
# These lines are commented to test the other algorithms for 4x4 or higher puzzles, since the run time of this cell would be too high.

#bfs: BFS = BFS(PuzzleState(initial_board))
#bfs_start_time: float = time.time()
#bfs.run()
#bfs_end_time: float = time.time()
#
#bfs.reconstruct_path()
#len(bfs.path), bfs.goal_state.cost, bfs_end_time - bfs_start_time

In [153]:
# These lines are commented to test the other algorithms for 4x4 or higher puzzles, since the run time of this cell would be too high.

#dfs: DFS = DFS(PuzzleState(initial_board))
#dfs_start_time: float = time.time()
#dfs.run()
#dfs_end_time: float = time.time()
#
#dfs.reconstruct_path()
#len(dfs.path), dfs.goal_state.cost, dfs_end_time - dfs_start_time

In [154]:
# It's advised to not run this cell if you want to test the algorithmic solver with 6x6 or higher puzzles, since the run time for those puzzles would be too high.

astar: AStar = AStar(PuzzleState(initial_board))
astar_start_time: float = time.time()
astar.run()
astar_end_time: float = time.time()

astar.reconstruct_path()
len(astar.path), astar.goal_state.cost, astar_end_time - astar_start_time

AStar progress:   0%|          | 0/1 [00:00<?, ?it/s]

(988, 16123, 2.8321759700775146)

In [155]:
fast_solver: FastSolver = FastSolver(PuzzleState(initial_board))
fast_solver_start_time: float = time.time()
fast_solver.run()
fast_solver_end_time: float = time.time()

fast_solver.reconstruct_path()
len(fast_solver.path), fast_solver_end_time - fast_solver_start_time

FastSolver progress:   0%|          | 0/24 [00:00<?, ?numbers correctly placed/s]

(461, 0.06145930290222168)